In [1]:
import pandas as pd
import numpy as np


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

from bs4 import BeautifulSoup

import string
import re

import spacy

from sklearn.feature_extraction.text import HashingVectorizer, TfidfVectorizer, CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score


from sklearn.linear_model import LogisticRegressionCV, LinearRegression

In [16]:
driver = webdriver.Chrome(executable_path='./chromedriver/chromedriver.exe')

jobs = ['data-scientist', 'data-analyst', 'business-analyst','research-scientist','machine-learning-engineer']

raw = []
for job in jobs:
    
    url = 'https://www.seek.com.au/'+job+'-jobs/'

    driver.get(url)
    WebDriverWait(driver, 3)

    pages = 12
    
    
    for page in range(2,pages):
        ads = driver.find_elements_by_css_selector('._2iNL7wI')
        links = [ad.get_attribute('href') for ad in ads]
        #print(ads)
        for link in links[2:]:
            driver.get(link)
            WebDriverWait(driver, 3)
            title = driver.find_element_by_css_selector('h1').text
            raw_html = driver.page_source
            raw.append([title, raw_html])
            driver.back()
            WebDriverWait(driver, 3)
        driver.get(url+'?page='+str(page))
raw_df = pd.DataFrame(raw,columns=['job_title','raw_html'])

driver.close()

raw_df.to_csv(r'./raw_data.csv')

In [33]:
raw_df = pd.read_csv('./raw_data.csv')

In [34]:
raw_df.head()

,Unnamed: 0,job_title,raw_html
0,0,Data Scientist,"<html lang=""en"" prefix=""og: http://ogp.me/ns#""..."
1,1,Data Scientist,"<html lang=""en"" prefix=""og: http://ogp.me/ns#""..."
2,2,Data Scientist,"<html lang=""en"" prefix=""og: http://ogp.me/ns#""..."
3,3,Data Scientist,"<html lang=""en"" prefix=""og: http://ogp.me/ns#""..."
4,4,Data Scientist,"<html lang=""en"" prefix=""og: http://ogp.me/ns#""..."


In [35]:
raw_df.drop(['Unnamed: 0'],inplace=True,axis=1)

In [36]:
raw_df.head()

,job_title,raw_html
0,Data Scientist,"<html lang=""en"" prefix=""og: http://ogp.me/ns#""..."
1,Data Scientist,"<html lang=""en"" prefix=""og: http://ogp.me/ns#""..."
2,Data Scientist,"<html lang=""en"" prefix=""og: http://ogp.me/ns#""..."
3,Data Scientist,"<html lang=""en"" prefix=""og: http://ogp.me/ns#""..."
4,Data Scientist,"<html lang=""en"" prefix=""og: http://ogp.me/ns#""..."


In [6]:
#raw_df.raw_html = raw_df.raw_html.apply(lambda x: BeautifulSoup(x).text)

In [32]:
soup = BeautifulSoup(raw_df.raw_html[1])

In [41]:
Advertiser_name = soup.find(attrs={'data-automation':'advertiser-name'}).text

'Aurec'

In [47]:
Job_Description = soup.find(attrs={'data-automation':'mobileTemplate'}).text

In [49]:
Date = soup.find(attrs={'data-automation':'job-detail-date'}).text

In [75]:
soup.findChildren(attrs={'class':'lwHBT6d'})

[<strong class="lwHBT6d">Try the app</strong>,
 <span class="lwHBT6d"><h2 class="_22uAIq9">Data Scientist</h2></span>,
 <span class="lwHBT6d">10 Feb 2020</span>,
 <strong class="lwHBT6d">Sydney</strong>,
 <span class="lwHBT6d">$400- $600 per day</span>,
 <span class="lwHBT6d">Contract/Temp</span>,
 <strong class="lwHBT6d">Information &amp; Communication Technology</strong>,
 <span class="lwHBT6d"><h2>Share this role</h2></span>,
 <span class="lwHBT6d">10 Feb 2020</span>,
 <strong class="lwHBT6d">Sydney</strong>,
 <span class="lwHBT6d">$400- $600 per day</span>,
 <span class="lwHBT6d">Contract/Temp</span>,
 <strong class="lwHBT6d">Information &amp; Communication Technology</strong>]

In [44]:
raw_df.drop([435],inplace=True)

In [46]:
raw_df['job_description'] = raw_df.raw_html.apply(lambda x: BeautifulSoup(x).find(attrs={'data-automation':'mobileTemplate'}).text)

In [47]:
raw_df.job_description[raw_df.job_description.isna()]

Series([], Name: job_description, dtype: object)

In [90]:
raw_df.iloc[435,:]

job_title          SEEK - Australia's no. 1 jobs, employment, car...
raw_html           <html lang="en" prefix="og: http://ogp.me/ns#"...
job_description                                                 None
Name: 435, dtype: object

In [91]:
raw_df.job_title.unique()

array(['Data Scientist', 'Senior Data Scientist - Machine Learning',
       'Data Scientist, Analytics, Machine Learning - Data Scientist, Data Statistician',
       'Data Scientist - Banking',
       'Senior Data Scientist, Machine Learning - Sydney',
       'Data Scientist - FMCG',
       'Data Scientist/Engineer in Leadership Technology - Remote Working',
       'Data Scientist - Remediation Analytics',
       'Customer Data Analytics and Data Science Opportunities',
       'Data Scientist - SAS to Tableau Migration',
       'Data Scientist APS 6', 'DATA SCIENTIST',
       'Data Scientist with Top Fund Manager',
       'Research Fellow - Systems Scientist/Data Scientist',
       'Senior Data Scientist', 'Data Scientist - Retail Banking',
       'Data Scientist / Mathematician',
       'Senior Data Scientist - Time-Series (Contrct 700-800pd)',
       'CSIRO Postdoc in Machine Learning & Artificial Intelligence: Constraints',
       'Quant / Data Scientist | Algorithmic Trading Compan

In [44]:
soup

<html lang="en" prefix="og: http://ogp.me/ns#"><head>
<meta charset="utf-8"/>
<meta content="block-all-mixed-content" http-equiv="Content-Security-Policy"/>
<meta content="yes" name="apple-mobile-web-app-capable"/>
<meta content="SEEK" name="apple-mobile-web-app-title"/>
<meta content="yes" name="mobile-web-app-capable"/>
<meta content="MC4yMzY5ODcwNzI5NjIzMDgyNQ==" name="request-id"/>
<meta content="#0d3880" name="theme-color"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=1, user-scalable=no" name="viewport"/>
<link href="/static/shared-web/favicon-4e1897dfd0901e8a3bf7e604d3a90636.ico" rel="icon"/>
<link href="/static/shared-web/iphone-4f6e398cfa84b92f7e5940285f0e46e3.png" rel="apple-touch-icon"/>
<link href="/static/shared-web/ipad-1f9b1e947eb83f87aee575239b50a993.png" rel="apple-touch-icon" sizes="76x76"/>
<link href="/static/shared-web/iphoneRetina-406e4373c0e30e2038d3a3f50346fc8c.png" rel="apple-touch-icon" sizes="120x120"/>
<link href="/static/shared-web/ipa

In [48]:
raw_df['job_info'] = raw_df.raw_html.apply(lambda x: BeautifulSoup(x).findChildren(attrs={'class':'lwHBT6d'}))

In [49]:
def text_maker(x):
    lis = []
    for i in x:
        lis.append(i.text)
    return lis

In [50]:
raw_df['job_info_text'] = raw_df.job_info.apply(text_maker)

In [51]:
raw_df.head()

,job_title,raw_html,job_description,job_info,job_info_text
0,Data Scientist,"<html lang=""en"" prefix=""og: http://ogp.me/ns#""...",Our client is a global Oil and Gas industry le...,"[[Try the app], [[Data Scientist]], [11 Feb 20...","[Try the app, Data Scientist, 11 Feb 2020, Bri..."
1,Data Scientist,"<html lang=""en"" prefix=""og: http://ogp.me/ns#""...",Where you’ll be working:You will be working wi...,"[[Try the app], [[Data Scientist]], [10 Feb 20...","[Try the app, Data Scientist, 10 Feb 2020, Syd..."
2,Data Scientist,"<html lang=""en"" prefix=""og: http://ogp.me/ns#""...","At Foxtel, we’re bringing television to Austra...","[[Try the app], [[Data Scientist]], [7 Feb 202...","[Try the app, Data Scientist, 7 Feb 2020, Sydn..."
3,Data Scientist,"<html lang=""en"" prefix=""og: http://ogp.me/ns#""...",Permanent opportunity for a Data Scientist to ...,"[[Try the app], [[Data Scientist]], [7 Feb 202...","[Try the app, Data Scientist, 7 Feb 2020, Melb..."
4,Data Scientist,"<html lang=""en"" prefix=""og: http://ogp.me/ns#""...",The Opportunity: Findex has built a reputation...,"[[Try the app], [[Data Scientist]], [4 Feb 202...","[Try the app, Data Scientist, 4 Feb 2020, Melb..."


In [52]:
draft_df = raw_df.drop(['raw_html','job_info'],axis=1)

In [53]:
draft_df.head()

,job_title,job_description,job_info_text
0,Data Scientist,Our client is a global Oil and Gas industry le...,"[Try the app, Data Scientist, 11 Feb 2020, Bri..."
1,Data Scientist,Where you’ll be working:You will be working wi...,"[Try the app, Data Scientist, 10 Feb 2020, Syd..."
2,Data Scientist,"At Foxtel, we’re bringing television to Austra...","[Try the app, Data Scientist, 7 Feb 2020, Sydn..."
3,Data Scientist,Permanent opportunity for a Data Scientist to ...,"[Try the app, Data Scientist, 7 Feb 2020, Melb..."
4,Data Scientist,The Opportunity: Findex has built a reputation...,"[Try the app, Data Scientist, 4 Feb 2020, Melb..."


In [54]:
#draft_df.to_csv('./draft_df.csv')

In [393]:
draft_df = pd.read_csv('./draft_df.csv')

In [394]:
def cleaner(elem):
    elem = elem.split(',')
    del elem[0]
    return elem

In [395]:
draft_df.job_info_text = draft_df.job_info_text.apply(lambda x: cleaner(x))

In [396]:
draft_df.head()

,Unnamed: 0,job_title,job_description,job_info_text
0,0,Data Scientist,Our client is a global Oil and Gas industry le...,"[ 'Data Scientist', '11 Feb 2020', 'Brisbane..."
1,1,Data Scientist,Where you’ll be working:You will be working wi...,"[ 'Data Scientist', '10 Feb 2020', 'Sydney',..."
2,2,Data Scientist,"At Foxtel, we’re bringing television to Austra...","[ 'Data Scientist', '7 Feb 2020', 'Sydney', ..."
3,3,Data Scientist,Permanent opportunity for a Data Scientist to ...,"[ 'Data Scientist', '7 Feb 2020', 'Melbourne..."
4,4,Data Scientist,The Opportunity: Findex has built a reputation...,"[ 'Data Scientist', '4 Feb 2020', 'Melbourne..."


In [397]:
draft_df.job_info_text[0][0]

" 'Data Scientist'"

In [398]:
draft_df['title'] = draft_df.job_info_text.apply(lambda x: x.pop(0))

In [399]:
draft_df['data_listed'] = draft_df.job_info_text.apply(lambda x: x.pop(0))

In [400]:
draft_df['location'] = draft_df.job_info_text.apply(lambda x: x.pop(0))

In [401]:
draft_df.tail()

,Unnamed: 0,job_title,job_description,job_info_text,title,data_listed,location
946,947,.NET WPF Developer | Eastern suburbs,The CompanyRobert Half Technology are working ...,"[ '(inc. super)', 'Contract/Temp', 'Informat...",'.NET WPF Developer | Eastern suburbs ','17 Jan 2020','Melbourne'
947,948,Google Cloud Architect- Sydney,Our client is a global leading financial servi...,"[ 'Competitive market rates', 'Contract/Temp'...",'Google Cloud Architect- Sydney','22 Jan 2020','Sydney'
948,949,Executive Search Consultant - Research & Highe...,Robertson Search is part of a 15-year-old priv...,"[ '$80, 000 - $89, 999', 'Full Time', 'Human...",'Executive Search Consultant - Research & Hig...,'30 Jan 2020','Melbourne'
949,950,PhD Scholarship: Remote monitoring of high-spe...,The Research Project Australia is the world le...,"[ 'Full Time', 'Education & Training', 'Shar...",'PhD Scholarship: Remote monitoring of high-s...,'24 Jan 2020','Hobart'
950,951,Full Stack Developer - .Net Core ($90-$140),THE ROLEThe role will be part of our engineeri...,"[ '($90-$140)', 'Full Time', 'Information & ...",'Full Stack Developer - .Net Core ($90-$140)','31 Jan 2020','Sydney'


In [402]:
draft_df.drop(['title'],inplace=True,axis=1)

In [403]:
draft_df.drop(['Unnamed: 0'],inplace=True,axis=1)

In [404]:
draft_df['salary'] = draft_df.job_info_text.apply(lambda x: x.pop(0))

In [405]:
draft_df.salary = draft_df.salary.apply(lambda x: x.strip("' '"))

In [406]:
def findsalary(x):
    salary = re.findall(pattern=r'\$(\d*\w)',string=x)
    try:
        return salary
    except:
        return np.nan

In [407]:
findsalary(draft_df.salary[0])

[]

In [408]:
draft_df.salary = draft_df.salary.apply(findsalary)

In [409]:
draft_df.head()

,job_title,job_description,job_info_text,data_listed,location,salary
0,Data Scientist,Our client is a global Oil and Gas industry le...,"[ 'Information & Communication Technology', '...",'11 Feb 2020','Brisbane',[]
1,Data Scientist,Where you’ll be working:You will be working wi...,"[ 'Contract/Temp', 'Information & Communicati...",'10 Feb 2020','Sydney',"[400, 600]"
2,Data Scientist,"At Foxtel, we’re bringing television to Austra...","[ 'Information & Communication Technology', '...",'7 Feb 2020','Sydney',[]
3,Data Scientist,Permanent opportunity for a Data Scientist to ...,"[ 'Information & Communication Technology', '...",'7 Feb 2020','Melbourne',[]
4,Data Scientist,The Opportunity: Findex has built a reputation...,"[ 'Science & Technology', 'Share this role', ...",'4 Feb 2020','Melbourne',[]


In [410]:
draft_df.salary

0              []
1      [400, 600]
2              []
3              []
4              []
          ...    
946            []
947            []
948          [80]
949            []
950     [90, 140]
Name: salary, Length: 951, dtype: object

In [411]:
# def crapsorter(x):
#     if x == [None]:
#         x = np.nan
#     elif len(x)==1:
#         try:
#             x = int(x)
#         except:
#             x = str(x)

In [412]:
#draft_df.salary = draft_df.salary.apply(crapsorter)

In [413]:
draft_df.tail()

,job_title,job_description,job_info_text,data_listed,location,salary
946,.NET WPF Developer | Eastern suburbs,The CompanyRobert Half Technology are working ...,"[ 'Contract/Temp', 'Information & Communicati...",'17 Jan 2020','Melbourne',[]
947,Google Cloud Architect- Sydney,Our client is a global leading financial servi...,"[ 'Contract/Temp', 'Information & Communicati...",'22 Jan 2020','Sydney',[]
948,Executive Search Consultant - Research & Highe...,Robertson Search is part of a 15-year-old priv...,"[000 - $89, 999', 'Full Time', 'Human Resour...",'30 Jan 2020','Melbourne',[80]
949,PhD Scholarship: Remote monitoring of high-spe...,The Research Project Australia is the world le...,"[ 'Education & Training', 'Share this role', ...",'24 Jan 2020','Hobart',[]
950,Full Stack Developer - .Net Core ($90-$140),THE ROLEThe role will be part of our engineeri...,"[ 'Full Time', 'Information & Communication T...",'31 Jan 2020','Sydney',"[90, 140]"


In [414]:
draft_df = draft_df[draft_df['salary'].map(lambda d: len(d)) > 0]

In [415]:
draft_df.head()

,job_title,job_description,job_info_text,data_listed,location,salary
1,Data Scientist,Where you’ll be working:You will be working wi...,"[ 'Contract/Temp', 'Information & Communicati...",'10 Feb 2020','Sydney',"[400, 600]"
5,Data Scientist,\n\n\n\n\n\n Data Scientist | Python | Strong ...,"[ 'Full Time', 'Information & Communication T...",'5 Feb 2020','Sydney',[130k]
9,Data Scientist,About the company My client is a company that...,"[000 + Superannuation', 'Full Time', 'Scienc...",'30 Jan 2020','Melbourne',[160]
12,Data Scientist,About Them: This tech start up are on a mis...,"[000-$100, 000 + Super', 'Full Time', 'Scien...",'3 Feb 2020','Sydney',[80]
17,Data Scientist,A brand new and exciting 6 month contract has ...,"[ 'Contract/Temp', 'Science & Technology', '...",'4 Feb 2020','Melbourne',"[700, 900]"


In [416]:
draft_df.drop(['job_info_text'],inplace=True,axis=1)

In [417]:
draft_df.reset_index(drop=True,inplace=True)

In [418]:
# if salary contains k replace with 000
# if salary length is more than 1 calculate average
# if salary is between 300 and 1000: salary*365
# if salary is less than 300: salary times 100
# convert all salarys to float

In [419]:
def bingbong(x):
    for i in x:
        
        i = float(i.lower().replace('k','000'))
        if (i > 300) & (i < 1000):
            i *= 365
        elif (i>100)&(i < 300):
            i *= 1000
        elif i <=100:
            i *= 42*52
        #except:
        return i 

In [420]:
draft_df.head()

,job_title,job_description,data_listed,location,salary
0,Data Scientist,Where you’ll be working:You will be working wi...,'10 Feb 2020','Sydney',"[400, 600]"
1,Data Scientist,\n\n\n\n\n\n Data Scientist | Python | Strong ...,'5 Feb 2020','Sydney',[130k]
2,Data Scientist,About the company My client is a company that...,'30 Jan 2020','Melbourne',[160]
3,Data Scientist,About Them: This tech start up are on a mis...,'3 Feb 2020','Sydney',[80]
4,Data Scientist,A brand new and exciting 6 month contract has ...,'4 Feb 2020','Melbourne',"[700, 900]"


In [421]:
bingbong(draft_df.salary[4])

255500.0

In [422]:
draft_df.salary = draft_df.salary.apply(bingbong)

In [423]:
draft_df.salary.unique()

array([146000., 130000., 160000., 174720., 255500.,  80000., 110000.,
       125000., 100000., 140000., 185640., 292000., 120000., 152880.,
       170000., 135000., 214032., 120120., 138000., 203112., 150000.,
       218400., 113000., 196560., 109200., 310250., 183456.,  83700.,
        75000., 131040.,  77000., 200750., 163800., 219000., 234695.,
       148512., 182500.,  60000., 226300., 262800., 255865., 135408.,
       248200., 273750., 127000.,  97000., 310615., 304775., 120450.,
       237250., 105000.,      0.,   1000., 112000., 133224., 179088.,
         4094.,  65000.,  75669., 104832., 102648., 157248., 144144.,
       187824., 113568., 211848., 159432., 103000., 192192., 205296.,
       216216.,  96564.,  55000., 115000.,  94000.,  63000., 155064.,
       328500.,  90000., 230000.,  98000., 102000.])

In [424]:
draft_df.salary[draft_df.salary == 0]

129    0.0
Name: salary, dtype: float64

In [427]:
draft_df.drop(129,inplace=True)

In [428]:
draft_df.salary[draft_df.salary == 0]

Series([], Name: salary, dtype: float64)

In [429]:
draft_df.salary[draft_df.salary == 4094]

148    4094.0
150    4094.0
Name: salary, dtype: float64

In [432]:
draft_df.iloc[147:150]

,job_title,job_description,data_listed,location,salary
148,Clinical Measurement Scientist - Senior (Vascu...,Metro South Health is Australia's first digita...,'7 Feb 2020','Brisbane',4094.0
149,Biology Programme Manager,Position: Biology Programme Manager at MecRx ...,'31 Jan 2020','Melbourne',174720.0
150,Clinical Measurement Scientist - Senior (Vascu...,Metro South Health is Australia's first digita...,'7 Feb 2020','Brisbane',4094.0


In [443]:
draft_df[draft_df.duplicated(keep=False)].sort_values('job_description')

,job_title,job_description,data_listed,location,salary
1,Data Scientist,\n\n\n\n\n\n Data Scientist | Python | Strong ...,'5 Feb 2020','Sydney',130000.0
238,Data Scientist,\n\n\n\n\n\n Data Scientist | Python | Strong ...,'5 Feb 2020','Sydney',130000.0
92,Data Scientist,\n\n\n\n\n\n Data Scientist | Python | Strong ...,'5 Feb 2020','Sydney',130000.0
242,LECTURER/SENIOR LECTURER IN MACHINE LEARNING (...,\nJob no: G - 0049924\nWork type: Continuing\n...,'21 Jan 2020','Melbourne',102000.0
243,LECTURER/SENIOR LECTURER IN MACHINE LEARNING (...,\nJob no: G - 0049924\nWork type: Continuing\n...,'21 Jan 2020','Melbourne',102000.0
...,...,...,...,...,...
56,Data Analytics Specialist,\nUQJobs No: 508695\nProperty and Facilities\...,'7 Feb 2020','Brisbane',77000.0
54,Data Analytics Specialist,\nUQJobs No: 508695\nProperty and Facilities\...,'7 Feb 2020','Brisbane',77000.0
83,Data Analytics Specialist,\nUQJobs No: 508695\nProperty and Facilities\...,'7 Feb 2020','Brisbane',77000.0
41,Big Data Engineer,Equifax in North Sydney are looking for a ta...,'29 Jan 2020','Sydney',110000.0


In [444]:
df_1 = draft_df.drop_duplicates()

In [445]:
df_1.shape

(208, 5)

In [2]:
df_1.to_csv('./df_1.csv',index=False)

NameError: name 'df_1' is not defined

In [16]:
df_1 = pd.read_csv('./df_1.csv')

In [17]:
df_1

,job_title,job_description,data_listed,location,salary
0,Data Scientist,Where you’ll be working:You will be working wi...,'10 Feb 2020','Sydney',146000.0
1,Data Scientist,\n\n\n\n\n\n Data Scientist | Python | Strong ...,'5 Feb 2020','Sydney',130000.0
2,Data Scientist,About the company My client is a company that...,'30 Jan 2020','Melbourne',160000.0
3,Data Scientist,About Them: This tech start up are on a mis...,'3 Feb 2020','Sydney',174720.0
4,Data Scientist,A brand new and exciting 6 month contract has ...,'4 Feb 2020','Melbourne',255500.0
...,...,...,...,...,...
203,Executive Search Consultant - Research & Highe...,Robertson Search is part of a 15-year-old priv...,'3 Feb 2020','Melbourne',174720.0
204,Reverse Engineer/Researcher,About the business GQR is working on an excl...,'15 Jan 2020','Sydney',150000.0
205,Big Data Engineer - Azure/AWS,About our Client: My client is one of the fa...,'17 Jan 2020','Melbourne',140000.0
206,Executive Search Consultant - Research & Highe...,Robertson Search is part of a 15-year-old priv...,'30 Jan 2020','Melbourne',174720.0


In [18]:
df_1.job_title.unique()

array(['Data Scientist', 'Data Scientist - Banking',
       'Data Scientist - Remediation Analytics', 'Data Scientist APS 6',
       'Data Scientist - Retail Banking',
       'Senior Data Scientist - Time-Series (Contrct 700-800pd)',
       'Data Scientist | Global Health Tech Business',
       'Data Scientist - Property and Retail',
       'Junior Marketing Data Scientist / Analyst',
       'Data Science Consultant', 'Data Science Engineer',
       'Data Analytics Consultants - Data Forensics - $70-90k',
       'Senior Data Scientist - $160k + Super + 20% bonus',
       'Data Engineer', 'Lead Data Scientist',
       'Software Engineer | Machine Learning & AI',
       'Research Scientist - Computational Platforms',
       'Manager Data Science and Data Visualisation',
       'Start your new career as a Data Analyst',
       'Machine Learning Engineer', 'Lead Data Science Analyst',
       'Associate Director Data and Analytics',
       'Data Science Software Engineer', 'Product Manager 

In [19]:
def remove_numbers(x):
    x = re.findall(pattern=r'\D+\b',string=x)
    return x

In [20]:
df_1.job_title = df_1.job_title.apply(remove_numbers)

In [21]:
df_1.job_description = df_1.job_description.apply(remove_numbers)

In [22]:
df_1.head()

,job_title,job_description,data_listed,location,salary
0,[Data Scientist],[Where you’ll be working:You will be working w...,'10 Feb 2020','Sydney',146000.0
1,[Data Scientist],[\n\n\n\n\n\n Data Scientist | Python | Strong...,'5 Feb 2020','Sydney',130000.0
2,[Data Scientist],[About the company My client is a company tha...,'30 Jan 2020','Melbourne',160000.0
3,[Data Scientist],[ About Them: This tech start up are on a mi...,'3 Feb 2020','Sydney',174720.0
4,[Data Scientist],"[A brand new and exciting , month contract ha...",'4 Feb 2020','Melbourne',255500.0


In [23]:
nlp = spacy.load("en_core_web_sm")

In [24]:
temp = df_1.job_description[0]

In [25]:
temp

['Where you’ll be working:You will be working with a huge brand name company that have major plans for ',
 '! This company has huge amounts of data at their disposal and a team that focuses on using the latest open source tools in the market!This is a ',
 " month initial contract with extension options.This company has very cool products that are used by millions across Australia! Who Are you?You will come from a strong Data Science background with solid coding ability built up through extensive commercial experience.This role will see you work with stakeholders to plan and design analytics projects. You will be working across a range of tech stacks to create insights and models that improve the performance of this company's unique products. Skills Required:At least ",
 ' yrs experience working as a Data Scientist with Big Data setsAble to code in either: Python or RStrong SQL skillsAWS - ',
 ', Redshift etcGoogle AnalyticsPhD would be favourable (Machine Learning, Stats, Maths, Comput

In [26]:
' '.join(temp)

"Where you’ll be working:You will be working with a huge brand name company that have major plans for  ! This company has huge amounts of data at their disposal and a team that focuses on using the latest open source tools in the market!This is a   month initial contract with extension options.This company has very cool products that are used by millions across Australia! Who Are you?You will come from a strong Data Science background with solid coding ability built up through extensive commercial experience.This role will see you work with stakeholders to plan and design analytics projects. You will be working across a range of tech stacks to create insights and models that improve the performance of this company's unique products. Skills Required:At least   yrs experience working as a Data Scientist with Big Data setsAble to code in either: Python or RStrong SQL skillsAWS -  , Redshift etcGoogle AnalyticsPhD would be favourable (Machine Learning, Stats, Maths, Computer Science etc)Cl

In [27]:
df_1.job_description = df_1.job_description.apply(lambda x: ' '.join(x))

In [28]:
type(df_1.job_description[0])

str

In [29]:
df_1.job_title = df_1.job_title.apply(lambda x: ' '.join(x))

In [31]:
df_1.job_title

0                                         Data Scientist
1                                         Data Scientist
2                                         Data Scientist
3                                         Data Scientist
4                                         Data Scientist
                             ...                        
203    Executive Search Consultant - Research & Highe...
204                          Reverse Engineer/Researcher
205                        Big Data Engineer - Azure/AWS
206    Executive Search Consultant - Research & Highe...
207               Full Stack Developer - .Net Core ($ -$
Name: job_title, Length: 208, dtype: object

In [32]:
df_1.to_csv('./clean_df.csv',index=False)

In [14]:
df_1['job_des_pars'] = df_1.job_description.apply(nlp)

In [15]:
df_1['job_tit_pars'] = df_1.job_title.apply(nlp)

In [30]:
temp = df_1.job_des_pars[0]

In [88]:
vect = TfidfVectorizer(ngram_range=(1,5),max_features=4000)

In [89]:
X_temp = vect.fit_transform(df_1.job_description)

In [90]:
X_temp

<208x4000 sparse matrix of type '<class 'numpy.float64'>'
	with 59716 stored elements in Compressed Sparse Row format>

In [91]:
y_temp = df_1.salary

In [92]:
lin_reg = LinearRegression()

In [93]:
lin_reg.fit(X_temp, y_temp)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [94]:
from sklearn.model_selection import cross_val_score

In [95]:
score = cross_val_score(lin_reg,X_temp,y_temp)

In [96]:
score.mean()

0.0672008390301591

In [97]:
df_1.location.unique()

array([" 'Sydney'", " 'Melbourne'", " 'ACT'", " 'Perth'", " 'Adelaide'",
       " 'Brisbane'", " 'Horsham & Grampians'"], dtype=object)

In [99]:
location_dums = pd.get_dummies(df_1.location,drop_first=True)

In [100]:
df_2 = df_1.join(location_dums)

In [123]:
df_2

,job_title,job_description,salary,job_des_pars,job_tit_pars,'Adelaide','Brisbane','Horsham & Grampians','Melbourne','Perth','Sydney'
0,Data Scientist,Where you’ll be working:You will be working wi...,146000.0,"(Where, you, ’ll, be, working, :, You, will, b...","(Data, Scientist)",0,0,0,0,0,1
1,Data Scientist,\n\n\n\n\n\n Data Scientist | Python | Strong ...,130000.0,"(\n\n\n\n\n\n , Data, Scientist, |, Python, |,...","(Data, Scientist)",0,0,0,0,0,1
2,Data Scientist,About the company My client is a company that...,160000.0,"(About, the, company, , My, client, is, a, co...","(Data, Scientist)",0,0,0,1,0,0
3,Data Scientist,About Them: This tech start up are on a mis...,174720.0,"( , About, Them, :, This, tech, start, up, a...","(Data, Scientist)",0,0,0,0,0,1
4,Data Scientist,A brand new and exciting month contract has ...,255500.0,"(A, brand, new, and, exciting, , month, cont...","(Data, Scientist)",0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
203,Executive Search Consultant - Research & Highe...,Robertson Search is part of a -year-old priva...,174720.0,"(Robertson, Search, is, part, of, a, , -year,...","(Executive, Search, Consultant, -, Research, &...",0,0,0,1,0,0
204,Reverse Engineer/Researcher,About the business GQR is working on an excl...,150000.0,"(About, the, business, , GQR, is, working, o...","(Reverse, Engineer, /, Researcher)",0,0,0,0,0,1
205,Big Data Engineer - Azure/AWS,About our Client: My client is one of the fa...,140000.0,"(About, our, Client, :, , My, client, is, on...","(Big, Data, Engineer, -, Azure, /, AWS)",0,0,0,1,0,0
206,Executive Search Consultant - Research & Highe...,Robertson Search is part of a -year-old priva...,174720.0,"(Robertson, Search, is, part, of, a, , -year,...","(Executive, Search, Consultant, -, Research, &...",0,0,0,1,0,0


In [101]:
df_2.head()

,job_title,job_description,data_listed,location,salary,job_des_pars,job_tit_pars,'Adelaide','Brisbane','Horsham & Grampians','Melbourne','Perth','Sydney'
0,Data Scientist,Where you’ll be working:You will be working wi...,'10 Feb 2020','Sydney',146000.0,"(Where, you, ’ll, be, working, :, You, will, b...","(Data, Scientist)",0,0,0,0,0,1
1,Data Scientist,\n\n\n\n\n\n Data Scientist | Python | Strong ...,'5 Feb 2020','Sydney',130000.0,"(\n\n\n\n\n\n , Data, Scientist, |, Python, |,...","(Data, Scientist)",0,0,0,0,0,1
2,Data Scientist,About the company My client is a company that...,'30 Jan 2020','Melbourne',160000.0,"(About, the, company, , My, client, is, a, co...","(Data, Scientist)",0,0,0,1,0,0
3,Data Scientist,About Them: This tech start up are on a mis...,'3 Feb 2020','Sydney',174720.0,"( , About, Them, :, This, tech, start, up, a...","(Data, Scientist)",0,0,0,0,0,1
4,Data Scientist,A brand new and exciting month contract has ...,'4 Feb 2020','Melbourne',255500.0,"(A, brand, new, and, exciting, , month, cont...","(Data, Scientist)",0,0,0,1,0,0


In [102]:
df_2.drop(['location'],inplace=True, axis=1)

In [103]:
df_2.drop(['data_listed'],inplace=True, axis=1)

In [113]:
from spacy.attrs import LOWER, POS, ENT_TYPE, IS_ALPHA
       

In [104]:
temp = df_2.job_des_pars[1]

In [114]:

np_array = temp.to_array([LOWER, POS, ENT_TYPE, IS_ALPHA])

In [116]:
np_array.shape

(283, 4)

In [ ]:
df_2['job_des_pars'].iloc[0]

In [133]:
dir(df_2['job_des_pars'].iloc[0])

['_',
 '__bytes__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__pyx_vtable__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '_bulk_merge',
 '_py_tokens',
 '_realloc',
 '_vector',
 '_vector_norm',
 'cats',
 'char_span',
 'count_by',
 'doc',
 'ents',
 'extend_tensor',
 'from_array',
 'from_bytes',
 'from_disk',
 'get_extension',
 'get_lca_matrix',
 'has_extension',
 'has_vector',
 'is_nered',
 'is_parsed',
 'is_sentenced',
 'is_tagged',
 'lang',
 'lang_',
 'mem',
 'merge',
 'noun_chunks',
 'noun_chunks_iterator',
 'print_tree',
 'remove_extension',
 'retokenize',
 'sentiment',
 'sents',
 'set_extension',
 'similarity',
 'tensor',
 'text',
 'text_with_ws',
 'to_array',
 'to_byte

In [132]:
df_2['job_des_pars'].apply(lambda x: list(x.sents))

0      (Where, you, ’ll, be, working, :, You, will, b...
1      (\n\n\n\n\n\n , Data, Scientist, |, Python, |,...
2                               (About, the, company,  )
3                                     (   , About, Them)
4      (A, brand, new, and, exciting,   , month, cont...
                             ...                        
203    (Robertson, Search, is, part, of, a,  , -year,...
204    (About, the, business,   , GQR, is, working, o...
205                          (About, our, Client, :,   )
206    (Robertson, Search, is, part, of, a,  , -year,...
207    (THE, ROLEThe, role, will, be, part, of, our, ...
Name: job_des_pars, Length: 208, dtype: object

In [128]:
df_2['job_des_pars'].apply(lambda x: x.text)

0      Where you’ll be working:You will be working wi...
1      \n\n\n\n\n\n Data Scientist | Python | Strong ...
2      About the company  My client is a company that...
3         About Them: This tech start up are on a mis...
4      A brand new and exciting   month contract has ...
                             ...                        
203    Robertson Search is part of a  -year-old priva...
204    About the business   GQR is working on an excl...
205    About our Client:   My client is one of the fa...
206    Robertson Search is part of a  -year-old priva...
207    THE ROLEThe role will be part of our engineeri...
Name: job_des_pars, Length: 208, dtype: object

In [134]:
job_description_vecs = df_2['job_des_pars'].apply(lambda x: pd.Series(x.vector))

In [145]:
score = cross_val_score(lin_reg,job_description_vecs,y_temp)
score.mean()

-2.000453745075664

In [147]:
sentence_vecs = df_2['job_des_pars'].apply(lambda x: pd.Series(x.sents))

In [181]:
# Lets use the stop_words argument to remove words like "and, the, a"
cvec = CountVectorizer(stop_words='english',ngram_range=(1,4),max_features=2000,min_df=3)

# Fit our vectorizer using our train data
cvec.fit(df_2.job_description)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=2000, min_df=3,
                ngram_range=(1, 4), preprocessor=None, stop_words='english',
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [182]:
# Transforming our x_train data using our fit cvec.
# And converting the result to a DataFrame.
X_train = pd.DataFrame(cvec.transform(df_2.job_description).todense(),
                       columns=cvec.get_feature_names())

In [183]:
# Which words appear the most?
word_counts = X_train.sum(axis=0)
word_counts.sort_values(ascending = False).head(20)

data           915
business       574
experience     539
research       397
work           375
role           314
team           292
skills         291
working        283
apply          255
science        221
development    205
information    195
position       182
learning       181
ability        165
analysis       161
opportunity    158
health         157
management     155
dtype: int64

In [184]:
y_temp.describe()

count       208.000000
mean     155698.019231
std       61727.957995
min        1000.000000
25%      113426.000000
50%      140000.000000
75%      196560.000000
max      328500.000000
Name: salary, dtype: float64

In [185]:
cat_y = pd.qcut(y_temp,6,labels=[0,1,2,3,4,5])

In [186]:
cat_y

0      3
1      2
2      3
3      3
4      5
      ..
203    3
204    3
205    2
206    3
207    4
Name: salary, Length: 208, dtype: category
Categories (6, int64): [0 < 1 < 2 < 3 < 4 < 5]

In [189]:
import matplotlib.pyplot as plt

In [191]:
from sklearn.model_selection import train_test_split

In [192]:
X_train, X_test, y_train, y_test = train_test_split(X_train,cat_y)

In [193]:
log_reg.fit(X_train, y_train)

LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False,
                     fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                     max_iter=10000, multi_class='auto', n_jobs=None,
                     penalty='l2', random_state=None, refit=True, scoring=None,
                     solver='lbfgs', tol=0.0001, verbose=0)

In [194]:
log_reg.score(X_test, y_test)

0.21153846153846154

In [195]:
from sklearn.model_selection import GridSearchCV

In [197]:
gs = GridSearchCV(cvec,param_grid={'ngram_range':[(1,2),(1,3),(1,4),(1,5),(1,6),(2,3),(2,4),(2,5),(2,6)],'max_features':[100,500,1000,2000]})

In [200]:
gs.fit(df_2.job_description,cat_y)

TypeError: If no scoring is specified, the estimator passed should have a 'score' method. The estimator CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=2000, min_df=3,
                ngram_range=(1, 4), preprocessor=None, stop_words='english',
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None) does not.